In [1]:
from langchain_core.documents import Document
documents =[
    Document(
        page_content="Dogs are great copanions, knon for thair loyalty and friendliness.",
        metadata={"source":"mammal-pets-doc"}
    ),
    Document(
        page_content="cats are independent pets that often enjoy their own space.",
        metadata={"source":"mammal-pets-doc"}
    ),
    Document(
        page_content="Goldfish are popular pets for beginners,requiring relatively simple care.",
        metadata={"source":"fish-pets-doc"}
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source":"bird-pets-doc"}
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source":"mammal-pets-doc"}
    )
]

In [2]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great copanions, knon for thair loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners,requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [3]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
llm=ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000206CA53DC70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000206CA53E630>, model_name='Llama3-8b-8192', groq_api_key=SecretStr('**********'))

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [7]:
from langchain_chroma import Chroma

vectorstore=Chroma.from_documents(documents,embedding=embeddings)
vectorstore

In [12]:
vectorstore.similarity_search("cats")[0]

Document(metadata={'source': 'mammal-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.')

In [15]:
await vectorstore.asimilarity_search("dog")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great copanions, knon for thair loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners,requiring relatively simple care.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [16]:
vectorstore.similarity_search_with_score('cat')

[(Document(metadata={'source': 'mammal-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.'),
  0.9351057410240173),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.5956902503967285),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great copanions, knon for thair loyalty and friendliness.'),
  1.5987904071807861),
 (Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.6657923460006714)]

### Retriever

In [17]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever=RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(['cat','dog'])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great copanions, knon for thair loyalty and friendliness.')]]

In [24]:
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)
retriever.batch(['cat','dog'])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great copanions, knon for thair loyalty and friendliness.')]]

In [27]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
message="""
Answer this question using the provided context only.

{question}

context:
{context}
"""
prompt=ChatPromptTemplate.from_messages([("human",message)])
rag_chain={"context":retriever,"question":RunnablePassthrough()}|prompt|llm
response=rag_chain.invoke("tell me about parrots")
print(response.content)

According to the provided context, parrots are:

* Intelligent birds
* Capable of mimicking human speech
